In [23]:
from bqplot import *
from bqplot import pyplot as plt 
import numpy as np

In [29]:
def _draw_mark(mark_type, options={}, axes_options={}, **kwargs):
    """Draw the mark of specified mark type.

    Parameters
    ----------
    mark_type: type
        The type of mark to be drawn
    options: dict (default: {})
        Options for the scales to be created. If a scale labeled 'x' is
        required for that mark, options['x'] contains optional keyword
        arguments for the constructor of the corresponding scale type.
    axes_options: dict (default: {})
        Options for the axes to be created. If an axis labeled 'x' is required
        for that mark, axes_options['x'] contains optional keyword arguments
        for the constructor of the corresponding axis type.
    figure: Figure or None
        The figure to which the mark is to be added.
        If the value is None, the current figure is used.
    cmap: list or string
        List of css colors, or name of bqplot color scheme
    """
    fig = kwargs.pop('figure', plt.current_figure())
    scales = kwargs.pop('scales', {})
    update_context = kwargs.pop('update_context', True)

    # Set the color map of the color scale
    cmap = kwargs.pop('cmap', None)
    if cmap is not None:
        # Add the colors or scheme to the color scale options
        options['color'] = dict(options.get('color', {}),
                                **_process_cmap(cmap))

    # Going through the list of data attributes
    for name in mark_type.class_trait_names(scaled=True):
        dimension = plt._get_attribute_dimension(name, mark_type)
        # TODO: the following should also happen if name in kwargs and
        # scales[name] is incompatible.
        if name not in kwargs:
            # The scaled attribute is not being passed to the mark. So no need
            # create a scale for this.
            continue
        elif name in scales:
            if update_context:
                plt._context['scales'][dimension] = scales[name]
        # Scale has to be fetched from the context or created as it has not
        # been passed.
        elif dimension not in plt._context['scales']:
            # Creating a scale for the dimension if a matching scale is not
            # present in _context['scales']
            traitlet = mark_type.class_traits()[name]
            rtype = traitlet.get_metadata('rtype')
            dtype = traitlet.validate(None, kwargs[name]).dtype
            # Fetching the first matching scale for the rtype and dtype of the
            # scaled attributes of the mark.
            compat_scale_types = [
                    Scale.scale_types[key]
                    for key in Scale.scale_types
                    if Scale.scale_types[key].rtype == rtype and
                    np.issubdtype(dtype, Scale.scale_types[key].dtype)
                ]
            sorted_scales = sorted(compat_scale_types,
                                   key=lambda x: x.precedence)
            scales[name] = sorted_scales[-1](**options.get(name, {}))
            # Adding the scale to the context scales
            if update_context:
                plt._context['scales'][dimension] = scales[name]
        else:
            scales[name] = _context['scales'][dimension]

    mark = mark_type(scales=scales, **kwargs)
    #mark = None #this is turned on when you need to comment out the fig.mark
    plt._context['last_mark'] = mark
    fig.marks = [m for m in fig.marks] + [mark] #comment out fig.mark and if statement to take the dots away 
    if kwargs.get('axes', True):
        axes(mark, options=axes_options)
    return mark


In [30]:
plt.figure()

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), scale_…

In [31]:
def scatter(x, y, **kwargs):
    kwargs['x'] = x
    kwargs['y'] = y
    return _draw_mark(Scatter, **kwargs)

In [32]:
scatter([1, 2, 3], [2, 4, 6])

/opt/clasp/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: Deprecated in traitlets 4.1, use the instance .metadata dictionary directly, like x.metadata[key] or x.metadata.get(key, default)
/opt/clasp/lib/python3.7/site-packages/ipykernel_launcher.py:59: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.str_ == np.dtype(str).type`.


TypeError: 'module' object is not callable